In [76]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, fbeta_score, precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from skmultilearn.adapt import MLkNN
from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multioutput import MultiOutputClassifier


import pandas as pd
import json
import os
import re

import pickle
import nltk
from nltk.tokenize import word_tokenize

import gensim 
from gensim.models import Word2Vec
import gensim.downloader


import spacy
import matplotlib
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ast import literal_eval
from tqdm import tqdm

import sklearn.metrics
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [77]:
nltk.download('punkt')
tqdm.pandas()

[nltk_data] Downloading package punkt to /homes/lgf21/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [78]:
# Open Pickle without lemmatisation: 

with open('merged_data_no_duplicates.pickle', 'rb') as handle:
    (X_train_text, X_test_text, Y_train_tactic, Y_test_tactic, Y_train, Y_test) = pickle.load(handle)

# Feature Extraction:

In [79]:
# call methods for feature extraction and evaluation:
import sys
sys.path.append('../src')

from methods import feature_extraction, evaluation

In [80]:
X_train, X_test = feature_extraction('TfIdfVectorizer', X_train_text, X_test_text)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [31]:
X_test_text.head()

319     Duqu - Wikipedia Duqu From Wikipedia, the free...
336     Disk Content Wipe - Enterprise | MITRE ATT&CK\...
1077    Skip to main contentSkip to footerSkip to main...
486     Access Token Manipulation - Enterprise | MITRE...
371     SSH Agent Hijacking - Clockwork Skip to Conten...
Name: Text, dtype: object

In [81]:
# Train and test: First delete techniques less than 9 
# We fix the random state to have the same dataset in our different tests@

sv_classifier = OneVsRestClassifier(LinearSVC(penalty = 'l2', loss = 'squared_hinge', dual = False, max_iter = 1000, class_weight = 'balanced', random_state=42), n_jobs = 1)
sv_classifier.fit(X_train, Y_train)


OneVsRestClassifier(estimator=LinearSVC(class_weight='balanced', dual=False,
                                        random_state=42),
                    n_jobs=1)

In [82]:
Y_pred = pd.DataFrame(sv_classifier.predict(X_test), columns=Y_test.columns)

In [59]:
evaluation(Y_pred, Y_test)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


,metric,result
0,macro precision,0.523972
1,micro precision,0.544892
2,macro recall,0.364063
3,micro recall,0.404830
4,macro fscore,0.452901
5,micro fscore,0.509628


In [84]:
# Save as pickle technique performance: 

with open('postprocessing.pickle', 'wb') as handle:
    pickle.dump(sv_classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)

OSError: [Errno 122] Disk quota exceeded

# Open Tactic Performance: 

In [72]:
# Open Pickle: 

with open('tactic_model_no_lemma.pickle', 'rb') as handle:
    tactic_model = pickle.load(handle)

In [73]:
data = '../src/scraping/tactic_dataset.json'

In [74]:
with open(data) as file:
    tactic_data = json.load(file)

In [25]:
# Open Pickle: 

#with open('tactic_train_test.pickle', 'rb') as handle:
#    X_train, X_test, Y_train_tactic, Y_test_tactic = pickle.load(handle)


In [189]:
Y_test_tactic_pickle

,TA0006,TA0002,TA0040,TA0003,TA0004,TA0008,TA0005,TA0010,TA0007,TA0009,TA0011,TA0001
319,0,0,0,0,0,0,1,0,0,0,0,0
336,0,0,1,0,0,0,0,0,0,0,0,0
1077,0,1,0,1,1,0,1,0,1,0,0,0
486,0,0,0,0,1,0,1,0,0,0,0,0
371,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1477,0,0,0,1,1,0,0,0,0,0,0,0
1376,0,1,0,1,1,0,0,0,0,0,0,0
415,0,0,0,1,0,0,1,0,0,0,0,0
880,0,0,0,1,0,0,1,0,0,0,1,0


In [75]:
Y_test_pred_tactic = pd.DataFrame(tactic_model.predict(X_test_tactic_pickle), columns = Y_test_tactic.columns)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 0000000
- 0000000010
- 0000000020
- 0000000030
- 0000000040
- ...
Feature names seen at fit time, yet now missing:
- agesen
- alors
- ampleur
- anciennes
- attaquants
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 74675 features, but LinearSVC is expecting 69856 features as input.

# Hanging Node:

In [70]:
# list of tactics and list of techniques:
from sklearn.preprocessing import MinMaxScaler

In [71]:
tactic_prob = pd.DataFrame(tactic_model.decision_function(X_test), columns=Y_test_tactic.columns)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- agesen
- alors
- ampleur
- anciennes
- attaquants
- ...
Feature names seen at fit time, yet now missing:
- 0000000
- 0000000010
- 0000000020
- 0000000030
- 0000000040
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 69856 features, but LinearSVC is expecting 74675 features as input.

In [ ]:
# if scaling at (-1,1) then threshold of .5 
# scaling determines improvement 
# (0-1) performs poorly with threshold of .5 

technique_prob = pd.DataFrame(MinMaxScaler((-1, 1)).fit_transform(sv_classifier.decision_function(X_test)), columns=Y_test.columns)

In [ ]:
def hanging_node(i, tactic_proba, technique_proba,tactic_threshold=0.5, technique_threshold=0.5, a = 1,b = 0, c=1, d=0):
    assert a > technique_threshold and c > technique_threshold and b < tactic_threshold and d < tactic_threshold
    pred_tactics = [tactic for tactic in tactic_proba.columns if tactic_proba.iloc[i][tactic] > tactic_threshold]
    pred_techniques = [technique for technique in technique_proba.columns if technique_proba.iloc[i][technique] > technique_threshold]
    #return pred_tactics, pred_techniques
    print((pred_tactics, pred_techniques))
    for tactic in tactic_proba.columns:
        for technique in tactic_data[tactic]['Technique_ID'][0]:
            if technique in pred_techniques and tactic not in pred_tactics:
                if technique_proba.iloc[i][technique] > a and tactic_proba.iloc[i][tactic] > b:
                    pred_tactics.append(tactic)
                elif technique_proba.iloc[i][technique] < c and tactic_proba.iloc[i][tactic] < d:
                    pred_techniques.remove(technique)
    print(pred_tactics, pred_techniques)
    print('-----------------------------')
    return pred_tactics, pred_techniques
    

In [ ]:
Y_test_tactics_hanging_node = []
Y_test_techniques_hanging_node = []
for i in range(len(X_test)):
    pred_tactics, pred_techniques = hanging_node(i, tactic_prob, technique_prob,tactic_threshold=0, technique_threshold=0.5,  a = 0.55, b = -0.9, c = 0.95, d = -0.30)
    Y_test_tactics_hanging_node.append([1 if tactic in pred_tactics else 0 for tactic in Y_test_tactic.columns])
    Y_test_techniques_hanging_node.append([1 if technique in pred_techniques else 0 for technique in Y_test.columns])

Y_test_tactics_hanging_node = pd.DataFrame(Y_test_tactics_hanging_node, columns=Y_test_tactic.columns)
Y_test_techniques_hanging_node = pd.DataFrame(Y_test_techniques_hanging_node, columns=Y_test.columns)

In [ ]:
evaluation(Y_test_tactics_hanging_node, Y_test_tactic)

In [ ]:
evaluation(Y_test_techniques_hanging_node, Y_test)

# Train Model on Reports Tactics:

In [ ]:
# see if training on only techniques improves or not
# only train techniques on reports belonging to a techniques' tactic

In [183]:
# Key as technique, value as tactic.

from collections import defaultdict
tactic_to_technique = defaultdict(list)
for tactic in tactic_data:
    for technique in tactic_data[tactic]['Technique_ID'][0]:
        if tactic in Y_train_tactic.columns:
            tactic_to_technique[technique].append(tactic)

In [184]:
# check for matching between tactic and technique
# find technique that are not in dictionary. 
# for Y_r, find report that only contains that technique and find what tactics is has 

for technique in Y_train.columns:
    if technique not in tactic_to_technique:
        index = Y_r[(Y_r.sum(axis=1)==1) & (Y_r[technique] == 1)].index[0]
        tacics = [tactic for tactic in Y_train_tactic.columns if 
                       int(Y_r_tactic.loc[index, tactic])==1]
        tactic_to_technique[technique] = tacics
    
    

In [185]:
classifiers = []
for technique in Y_train.columns:
    indexes = Y_train_tactic[Y_train_tactic.apply(lambda x: any(x[tactic] ==1 for tactic in tactic_to_technique[technique]), axis=1)].index
    X_train_technique = X_train.loc[indexes]
    Y_train_technique = Y_train.loc[indexes, technique]
    clf = LinearSVC(penalty = 'l2', loss = 'squared_hinge', dual = False, max_iter = 1000, class_weight = 'balanced', random_state=42)
    clf.fit(X_train_technique, Y_train_technique)
    classifiers.append(clf)
    

In [199]:
Y_pred = pd.DataFrame(0, index=X_test.index, columns=Y_test.columns)
for i, technique in enumerate(Y_test.columns):
    indexes = Y_test_pred_tactic[Y_test_pred_tactic.apply(lambda x: any(x[tactic] ==1 for tactic in tactic_to_technique[technique]), axis=1)].index
    Y_pred.loc[indexes, technique] = classifiers[i].predict(X_test.loc[indexes])

In [201]:
evaluation(Y_pred, Y_test) # performance for technique 

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


,metric,result
0,macro precision,0.499174
1,micro precision,0.505065
2,macro recall,0.341547
3,micro recall,0.401380
4,macro fscore,0.429969
5,micro fscore,0.480253


# Linking Model:

In [56]:
data = '../src/scraping/tactic_dataset.json'

with open(data) as file:
    tactic_data = json.load(file)

In [60]:
df_tactic = pd.DataFrame(tactic_data).transpose()


In [65]:
df_tactic['Technique_ID'] = df_tactic['Technique_ID'].apply(lambda x: x[0])

In [67]:
df_tactic.columns

Index(['Tactic_ID', 'Link', 'Tactic_Name', 'Description', 'Technique_ID'], dtype='object')

In [68]:
df_tactic = df_tactic[['Tactic_ID', 'Technique_ID']]

In [69]:
df_tactic

,Tactic_ID,Technique_ID
TA0043,TA0043,"[T1595, T1592, T1589, T1590, T1591, T1598, T15..."
TA0042,TA0042,"[T1583, T1586, T1584, T1587, T1585, T1588, T1608]"
TA0001,TA0001,"[T1189, T1190, T1133, T1200, T1566, T1091, T11..."
TA0002,TA0002,"[T1059, T1609, T1610, T1203, T1559, T1106, T10..."
TA0003,TA0003,"[T1098, T1197, T1547, T1037, T1176, T1554, T11..."
TA0004,TA0004,"[T1548, T1134, T1547, T1037, T1543, T1484, T16..."
TA0005,TA0005,"[T1548, T1134, T1197, T1612, T1622, T1140, T16..."
TA0006,TA0006,"[T1557, T1110, T1555, T1212, T1187, T1606, T10..."
TA0007,TA0007,"[T1087, T1010, T1217, T1580, T1538, T1526, T16..."
TA0008,TA0008,"[T1210, T1534, T1570, T1563, T1021, T1091, T10..."


In [ ]:
# technique model
# Open Pickle: 

with open('tactic_model.pickle', 'rb') as handle:
    tactic_model = pickle.load(handle)